In [1]:
# coding=utf-8
import tensorflow as tf
import numpy as np
from __future__  import print_function
import pandas as pd
from HiddenMarkovModel import HiddenMarkovModel
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import manhattan_distances
%matplotlib inline

In [8]:
# p0 = np.array([0.6,0.4])
# emi = np.array([[0.5,0.1],
#                [0.4,0.3],
#                [0.1,0.6]])
# trans = np.array([[0.7,0.3],
#                  [0.4,0.6]])
# states = {0:'Healthy',1:'Fever'}

# obs = {0:'normal',1:'cold',2:'dizzy'}

# obs_seq = np.array([0,0,1,2,2])

# df_p0 = pd.DataFrame(p0,index=['Healthy','Fever'],columns=['Prob'])
# df_emi = pd.DataFrame(emi,index=['Normal','Cold','Dizzy'],columns=['Healthy','Fever'])
# df_trans = pd.DataFrame(trans,index=['fromHealthy','fromFever'],columns=['toHealthy','toFever'])



# model =  HiddenMarkovModel(trans, emi, p0)
# states_seq, state_prob = model.run_viterbi(obs_seq, summary=True)

# print(states_seq)

In [2]:
def pre_processing(filename):
    file_2g = pd.read_csv(filename).loc[:,['MRTime','Grid_center_x','Grid_center_y','RNCID_1', 'CellID_1','EcNo_1', 'RSCP_1', 'RNCID_2', 'CellID_2','EcNo_2', 'RSCP_2','RNCID_3', 'CellID_3','EcNo_3', 'RSCP_3', 'RNCID_4', 'CellID_4','EcNo_4', 'RSCP_4', 'RNCID_5', 'CellID_5','EcNo_5', 'RSCP_5', 'RNCID_6', 'CellID_6','EcNo_6', 'RSCP_6','Grid_ID']]
    gong_can = pd.read_csv("final_2g_gongcan.csv", encoding='gbk').loc[:,['CGI', 'LAC', 'CI', u'经度', u'纬度']]
    # merge_data = pd.merge(file_2g, gong_can, left_on=['RNCID_1','CellID_1'], right_on=['LAC','CI'], how='inner')[['MRTime','EcNo_1', 'RSCP_1', 'CGI', u'经度', u'纬度', 'Grid_ID']]

    for i in range(6):
        RNCID = "RNCID_" + str(i+1)
        CellID = "CellID_" + str(i+1)
        station = pd.merge(file_2g, gong_can, left_on=[RNCID, CellID], right_on=['LAC','CI'], how='inner')[['CGI']]
        file_2g[[str(i+1)+'station']] = station
        file_2g['RSSI'+str(i+1)] = file_2g['EcNo_' + str(i+1)] - file_2g['RSCP_' + str(i+1)]

    merge_data = file_2g
    
    merge_data['MRTime'] = merge_data['MRTime'].str.split()
    timeCol = pd.Series(map(lambda x:x[1],merge_data['MRTime']),index=merge_data.index)
    timeCol = pd.to_datetime(timeCol,format="%H:%M:%S.%f")
    merge_data['MRTime'] = timeCol
    
    merge_data = merge_data.sort_values(by=['MRTime'])
    
    for i in range(1,7):
        del merge_data['EcNo_' + str(i)],merge_data['RSCP_' + str(i)]
        
    for i in range(1,7):
        del merge_data['RNCID_' + str(i)],merge_data['CellID_' + str(i)]
        
    return merge_data


In [3]:
train_data = pre_processing('final_2g_tr.csv')
test_data = pre_processing('final_2g_te.csv')

In [4]:
max_rssi = train_data['RSSI1'].max()

In [48]:
def emi_score(f1, group, ru, rssi_num, dRmax):
    station_compared = []
    rssi_code = []
    for i in range(1, rssi_num + 1):
        station_compared.append(str(i) + 'station')
        rssi_code.append('RSSI' + str(i))
    # 非空基站
    ls = f1[station_compared].dropna().tolist()
    # 匹配的基站
    containIndex = group[station_compared].isin(ls).index
    # 计算欧式距离
    group['distance'] = euclidean_distances(group.loc[containIndex, rssi_code].values,
                                            np.mat(f1[rssi_code].values)).flatten()
    # 计算索引
    max_index = group['distance'].argmax()
    # 计算匹配基站数
    M = (~group.loc[max_index, station_compared].isnull()).sum()
    return M * ru + (dRmax - group.loc[max_index, 'distance'] / M) if M > 0 else 0

In [49]:
emi = pd.DataFrame(0,index = test_data.index,columns=sorted(train_data['Grid_ID'].dropna().unique()))

In [50]:
train_group = train_data.groupby(['Grid_ID'])

In [9]:
for row in range(0, len(test_data)):
    if row%1000 == 0:
        print(row)
    fnew = test_data.loc[row]
    for key, group in train_group:
        emi.loc[row, key] = emi_score(fnew, group, 1, 6, max_rssi)

In [5]:
manhatan = train_data[['Grid_center_x','Grid_center_y']]

In [6]:
manhatan = manhatan.fillna(0)

In [9]:
distance_matrix = manhattan_distances(manhatan,manhatan)

MemoryError: 

In [ ]:
trans = pd.DataFrame(distance_matirx,index=sorted(train_data['Grid_ID'].dropna().unique()),columns=sorted(train_data['Grid_ID'].dropna().unique()))
trans